In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

F:\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 读入数据,通过mnist自带方法input_data读入训练数据，并将标签转化为10位独热码

In [2]:
mnist = input_data.read_data_sets("MINST_data/", one_hot=True)

W0330 14:31:46.507869  8664 deprecation.py:323] From <ipython-input-2-043a7e136cf2>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0330 14:31:46.509864  8664 deprecation.py:323] From F:\Anaconda\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0330 14:31:46.511861  8664 deprecation.py:323] From F:\Anaconda\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement 

Extracting MINST_data/train-images-idx3-ubyte.gz


W0330 14:31:46.822114  8664 deprecation.py:323] From F:\Anaconda\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0330 14:31:46.824030  8664 deprecation.py:323] From F:\Anaconda\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0330 14:31:46.887873  8664 deprecation.py:323] From F:\Anaconda\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please 

Extracting MINST_data/train-labels-idx1-ubyte.gz
Extracting MINST_data/t10k-images-idx3-ubyte.gz
Extracting MINST_data/t10k-labels-idx1-ubyte.gz


In [3]:
mnist.train.labels[:10,:]   # 独热码

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

# 前向网络部分

In [4]:
x = tf.placeholder("float", [None, 784])

In [5]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [6]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

# 做交叉熵函数

In [7]:
y_ = tf.placeholder("float", [None,10])

In [8]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))    # 计算交叉熵

## 优化器

In [9]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)  # 优化交叉熵

In [10]:
# 初始化变量
init = tf.global_variables_initializer()

# 执行，创建Session()

In [11]:
sess = tf.Session()
sess.run(init)

# 执行训练

In [12]:
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# 评估模型

In [13]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [14]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9185


# 卷积

## 创建前向网络

In [15]:
x = tf.placeholder("float", [None, 784])

In [16]:
y_ = tf.placeholder("float", shape=[None, 10])

#  初始化

In [17]:
# 初始化w
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

# 初始化b
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

#  卷积和池化

In [18]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')    # strides 步长    padding 0边距   'SAME' 不变

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')    # x 池化的图

# 第一层卷积

In [20]:
W_conv1 = weight_variable([5, 5, 1, 32])   # 卷积大小5x5x1,通道数为1， 32个卷积  
b_conv1 = bias_variable([32])     # W矩阵的变量数为5*5*1*32,  b矩阵的变量为32

In [21]:
x_image = tf.reshape(x, [-1,28,28,1])   # 黑白图像通道数为1  

In [22]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)    #  卷积后激活函数

In [23]:
h_pool1 = max_pool_2x2(h_conv1)   # 池化

## 第二层卷积

In [24]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## 全连接

In [25]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## Dropout

In [27]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W0330 14:57:02.973510  8664 deprecation.py:506] From <ipython-input-27-ec987d39f99c>:2: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## 输出

In [28]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## 损失函数

In [29]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))

## 优化器

In [30]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

## 创建输出的量

In [31]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

## 执行

In [37]:
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    for i in range(20000):
      batch = mnist.train.next_batch(50)
      if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.16
step 100, training accuracy 0.22
step 200, training accuracy 0.1
step 300, training accuracy 0.18
step 400, training accuracy 0.22
step 500, training accuracy 0.08
step 600, training accuracy 0.16
step 700, training accuracy 0.16
step 800, training accuracy 0.28
step 900, training accuracy 0.18
step 1000, training accuracy 0.28
step 1100, training accuracy 0.24
step 1200, training accuracy 0.18
step 1300, training accuracy 0.18
step 1400, training accuracy 0.22
step 1500, training accuracy 0.36
step 1600, training accuracy 0.36
step 1700, training accuracy 0.36
step 1800, training accuracy 0.32
step 1900, training accuracy 0.5
step 2000, training accuracy 0.58
step 2100, training accuracy 0.48
step 2200, training accuracy 0.58
step 2300, training accuracy 0.44
step 2400, training accuracy 0.36
step 2500, training accuracy 0.66
step 2600, training accuracy 0.52
step 2700, training accuracy 0.52
step 2800, training accuracy 0.48
step 2900, training accuracy